In [17]:
from messages import Price, Trade

In [21]:
packet = "TYPE=TRADE|FEEDCODE=SP-FUTURE|SIDE=ASK|PRICE=3048.25|VOLUME=533"
price = Trade.from_packet(packet)
print(price.side)

ASK


In [11]:
# Get price from recent packet

# 2 queues of length 60
queue_ESX = Queue()
queue_SP = Queue()

# If price is a BID  
  # THEN  
  # If price is ESX  
    # If queue is not full 
      # add to queue
    # Else 
      # call buyOrSell



  # If price is SP 
    # If queue is not full 
      # add to queue  
    # Else 
      # call buyOrSell


def buyOrSell(queue, newval):
  # calculate mean based on queue
  # calculated std
  # calculate z-score
  # if z > 2 == negative value (sell)
  # else if z < -2 == positive value (buy)
  # return buy or sell and which price if so  
  # return 0 if nothing 


SyntaxError: unexpected EOF while parsing (<ipython-input-11-76235e31617b>, line 29)